In [1]:
%matplotlib inline
from d2l import tensorflow as d2l
import tensorflow as tf

In [2]:
X = d2l.normal(0.0, 1, (1000, 2))
A = d2l.tensor([[1, 2], [-0.1, 0.5]])
b = d2l.tensor([1, 2])
data = d2l.matmul(X, A) + b

InvalidArgumentError: Value for attr 'T' of float is not in the list of allowed values: int32, int64
	; NodeDef: {{node RandomStandardNormal}}; Op<name=RandomStandardNormal; signature=shape:T -> output:dtype; attr=seed:int,default=0; attr=seed2:int,default=0; attr=dtype:type,allowed=[DT_HALF, DT_BFLOAT16, DT_FLOAT, DT_DOUBLE]; attr=T:type,allowed=[DT_INT32, DT_INT64]; is_stateful=true> [Op:RandomStandardNormal]